In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('./vast2014geo.csv', index_col=0, parse_dates=['Timestamp'])
data.head()

,FullName,Timestamp,category,id,time
0,"Alcazar, Lucas",2014-01-06 07:00:00,Coffee,"Alcazar, Lucas@2014-01-06",7
1,"Alcazar, Lucas",2014-01-06 08:00:00,GASTech,"Alcazar, Lucas@2014-01-06",8
2,"Alcazar, Lucas",2014-01-06 09:00:00,GASTech,"Alcazar, Lucas@2014-01-06",9
3,"Alcazar, Lucas",2014-01-06 10:00:00,GASTech,"Alcazar, Lucas@2014-01-06",10
4,"Alcazar, Lucas",2014-01-06 11:00:00,GASTech,"Alcazar, Lucas@2014-01-06",11


In [3]:
from sven import StorylineChart

def show_storyline_for_one_person(data, person, dates=None, show_typical=True, location_granularity = 'category'):
    if dates is None:
        dates = data.Timestamp.unique()
        
    dates = set(map(pd.Timestamp, dates))

    typical = data[data.FullName == person]\
        .groupby(['FullName', 'time'])\
        [location_granularity]\
        .apply(lambda ser: ser.value_counts().idxmax())\
        .to_frame()\
        .reset_index()

    typical['id'] = typical.FullName + '@Typical'

    mask = (data.FullName == person) &\
        (np.any([data.Timestamp.dt.date == d for d in dates], axis=0))
    
    
    data = data[mask].drop(columns='Timestamp')
    
    if show_typical:
        data = pd.concat((typical, data))
    
    lineLabel = {
        v: v.split('@')[-1]
        for v in data.id.unique()
    }
    
    return StorylineChart(
        data=data,
        group=location_granularity,
        lineLabel=lineLabel
    )

show_storyline_for_one_person(
    data,
    'Alcazar, Lucas',
    dates=['2014-01-9']
)

StorylineChart(component='StorylineChart', props={'data': [{'FullName': 'Alcazar, Lucas', 'time': 0, 'category…